# Create json files for shared trips choropleth (Figs 3a, 3b)
Fig 3a - number of requested (and completed) shared trips  
Fig 3b - proportion of requested shared trips that were matched up  

Queries stored in `~/Documents/PROJECTS/VFH/COT_PAGE/QUERIES/` as:  
`Fig3a_prop_shared_trips_by_nn.sql`  
`Fig3b_prop_actual_shared_by_nn.sql`  

**refs**    

* https://geoffboeing.com/2015/10/exporting-python-data-geojson/  
Huom: `with open(output_filename, 'wb') as output_file:` should be `'w'` not `'wb'`  
* geoJSON minifier: https://jsonformatter.org/json-minify  

## Fig 3a Query (Fig3a_prop_shared_trips_by_nn.sql)
From https://github.com/CityofToronto/bdit_vfh/blob/master/notebooks/vfh_final.ipynb, **Proportion of Shared Ride Trips by Neighbourhood, September 2018, Exhibit 3-25**:

```
CREATE TABLE cnangini.fig3a_shared_trips AS
WITH daily_ave AS (

    SELECT sum(count) as count, pickup_neighbourhood 
    FROM ptc.trip_data_agg_neighbourhood_pool
    WHERE pickup_datetime > '2018-08-31'
    GROUP BY  pickup_neighbourhood
),

daily_ave_all AS (
    SELECT sum(count) as count, pickup_neighbourhood 
    FROM ptc.trip_data_agg_neighbourhood
    WHERE pickup_datetime > '2018-08-31'
    GROUP BY  pickup_neighbourhood
)

SELECT pickup_neighbourhood, ROUND(100 * a.count/b.count) AS prop, geom
FROM daily_ave a
LEFT JOIN daily_ave_all b USING (pickup_neighbourhood)
LEFT JOIN gis.neighbourhood ON pickup_neighbourhood = area_s_cd::integer
WHERE pickup_neighbourhood IS NOT NULL
ORDER by pickup_neighbourhood;

-- Add dummy rows for nn 141, 142
INSERT INTO cnangini.fig3a_shared_trips (pickup_neighbourhood, prop) 
VALUES (141, NULL);

INSERT INTO cnangini.fig3a_shared_trips (pickup_neighbourhood, prop) 
VALUES (142, NULL);

```

_**Note:** table `cnangini.fig3a_shared_trips` to be deleted from schema; saved as `~/Documents/PROJECTS/VFH/COT_PAGE/INTERIM_DATA/FROM_PGADMIN/Fig3a_shared_trips.csv`_

## Fig 3b Query (Fig3b_prop_actual_shared_by_nn.sql)
**made by Raph à-la-volée :**  

```
CREATE TABLE cnangini.fig3b_matched_shared_trips AS
 WITH raw_data AS (
     SELECT trip_data_new_geom.pickup_datetime::date AS pickup_datetime,
            trip_data_new_geom.pickup_neighbourhood,
            date_part('hour'::text, trip_data_new_geom.pickup_datetime) AS hr,
            trip_data_new_geom.ptctripid,
	 		count(*) OVER (PARTITION BY ptctripid) AS num_shared
       FROM ptc.trip_data_new_geom
      WHERE (trip_data_new_geom.provider_id = ANY (ARRAY[28608558, 1608409])) 
             AND trip_data_new_geom.dropoff_datetime IS NOT NULL 
             AND trip_data_new_geom.pickup_datetime IS NOT NULL
             AND trip_data_new_geom.pickup_datetime >= '2018-09-01 00:00:00'::timestamp without time zone 
             AND trip_data_new_geom.dropoff_datetime >= '2018-09-01 00:00:00'::timestamp without time zone
)

SELECT pickup_neighbourhood, 
   100 * SUM(CASE WHEN num_shared > 1 THEN 1 ELSE 0 END) --Count of picked up passengers in this neighbourhood who   actually shared their trip
   / COUNT(*) -- number of completed shared trip requests for this neighbourhood
   AS proportion_matched_shared_trips,
   geom
FROM raw_data
LEFT JOIN gis.neighbourhood ON pickup_neighbourhood = area_s_cd::integer
WHERE pickup_neighbourhood IS NOT NULL
GROUP BY pickup_neighbourhood, geom
ORDER BY pickup_neighbourhood;

-- Add dummy rows for nn 141, 142
INSERT INTO cnangini.fig3b_matched_shared_trips (pickup_neighbourhood, proportion_matched_shared_trips) 
VALUES (141, NULL);

INSERT INTO cnangini.fig3b_matched_shared_trips (pickup_neighbourhood, proportion_matched_shared_trips) 
VALUES (142, NULL);

```

_**Note:** table `cnangini.fig3b_matched_shared_trips` to be deleted from schema; saved as `~/Documents/PROJECTS/VFH/COT_PAGE/INTERIM_DATA/FROM_PGADMIN/Fig3b_matched_shared_trips.csv`_

# Save tables to csv file in pgAdmin

Go to pgAdmin and export `cnangini.fig3a_shared_trips` and `cnangini.fig3b_matched_shared_trips` to csv.  

=> save in `~/Documents/PROJECTS/VFH/COT_PAGE/INTERIM_DATA/FROM_PGADMIN/` as  
`Fig3a_shared_trips.csv` and  
`fig3b_matched_shared_trips.csv`  